## <i class="fa fa-repeat"></i> 切换pip软件源

In [ ]:
import os
import subprocess

def check_pip_list():
    output = !pip config list
    print(output)
    if "global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'" in output and "global.trusted-host='pypi.tuna.tsinghua.edu.cn'" in output:
        return True
    else:
        return False
if not check_pip_list():
    # 查找pip.conf文件
    pip_conf_file = os.popen("find / -name 'pip.conf'").read().strip()
    # 直接覆盖pip.conf文件
    with open(pip_conf_file, 'w') as f:
        f.write("[global]\nindex-url = https://pypi.tuna.tsinghua.edu.cn/simple\ntrusted-host = pypi.tuna.tsinghua.edu.cn")
        # f.write("")
    # 验证修改是否成功
    if check_pip_list():
        print("pip软件源切换成功！")
    else:
        print("pip软件源切换失败，请检查脚本是否正确运行。")
else:
    print("pip软件源本来就是清华源，无需切换")

In [ ]:
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/msys2/ 
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/ 
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/ 
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/peterjc123/
!conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/pytorch/
!conda config --set show_channel_urls yes

##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
import subprocess

libraries_to_install = [
    
]

tools_to_install = [
    "iputils-ping",
]

installed_libraries = subprocess.run(["pip", "freeze"], capture_output=True).stdout.decode().split("\n")

# 尝试安装所有未安装的库
for library in libraries_to_install:
    if library == "black[jupyter]" and any("black" in installed_library for installed_library in installed_libraries):
        print(f"【 {library} 】已经安装，跳过安装")
        continue
    elif not any(library in installed_library for installed_library in installed_libraries):
        !pip install $library
    else:
        print(f"【 {library} 】已经安装，跳过安装")
        
# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
from func.env import setProxy
cb=setProxy()
proxy=cb['proxy']
region=cb['region']

## <i class="fa fa-star"></i> Notebook环境切换
在JupyterLab的Notebook中使用新的Conda环境

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import ipywidgets as widgets

python_version = '3.10.6'

custom_env_name = 'py' + python_version

# custom_env_name = 'base'

s = getoutput('conda info --env')
print(s)
if custom_env_name not in s:
    print(f'当前系统中没有{custom_env_name}环境，正在安装')
    # 创建Conda新的虚拟环境
    # 构建一个虚拟环境
    # 更新bashrc中的环境变量
    # 在 /openbayes/home 下创建一个新的环境：https://openbayes.com/docs/gear/conda/#%E4%BD%BF%E7%94%A8-conda-%E5%88%9B%E5%BB%BA%E6%96%B0%E7%9A%84%E7%8E%AF%E5%A2%83
    python_install_args = [
        'source activate','&&',
        f'cd {root_dir}','&&',
        f'conda create -p {custom_env_name} python={python_version} -y','&&',
        'conda init bash','&&',
        'source /root/.bashrc -y'
    ]
    
    # !source /usr/local/etc/profile.d/conda.sh &&\
    # conda create -n $custom_env_name python=$python_version -y &&\
    # conda init bash && source /root/.bashrc -y
    
    python_install_cmd = (" ").join(python_install_args)
    !$python_install_cmd
    
else:
    print(f'已经安装了{custom_env_name}环境')

# clear_output(wait=True)
# print("Done!")

widgets.Button(
    value=False,
    description='环境部署成功!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

## 交代Base_args

In [ ]:
from IPython.display import clear_output
from func.env import setProxy,getWebUIDir
import subprocess

def find_conda_env(name):
    # Get a list of all conda environments on the system
    output = subprocess.run(["conda", "env", "list"], capture_output=True)
    envs = output.stdout.decode().split("\n")

    # Find the path of the environment
    for env in envs:
        if name in env:
            path = env.split()[-1]
            print(f"The path of the environment '{name}' is {path}")
            return path
    else:
        print(f"Environment '{name}' not found.")
        return None



try:
    proxy,region
except NameError:
    cb=setProxy()
    proxy=cb['proxy']
    region=cb['region']
    clear_output(wait=True)

webUIDir=getWebUIDir()
print(webUIDir)

custom_env_dir = find_conda_env(custom_env_name)

base_args = [
    'source activate','&&',
    f'conda activate {custom_env_dir}','&&',
    proxy,'&&'
       ]

## <i class="fa fa-bolt"></i> 安装Xformers和triton

大幅减少计算时间和显存占用，代价是生成的图每个都有点变化，强烈推荐安装!  
但不同核心代号的显卡需要不同的编译包，不能混用  
当然此笔记会帮你自动检测这台机器是否适合安装，而且会自动帮你安装合适的编译包

### 从数据集中查找并安装Xformers

In [ ]:
import subprocess,os,time
'''
执行!python -m xformers.info命令会返回类似以下格式的信息，请参考

xFormers 0.0.16+bc08bbc.d20230122
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.flshattF:               available
memory_efficient_attention.flshattB:               available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.tritonflashattF:        available
memory_efficient_attention.tritonflashattB:        available
swiglu.fused.p.cpp:                                available
is_triton_available:                               True
is_functorch_available:                            False
pytorch.version:                                   1.12.1+cu113
pytorch.cuda:                                      available
gpu.compute_capability:                            8.6
gpu.name:                                          NVIDIA GeForce RTX 3090
build.info:                                        available
build.cuda_version:                                1103
build.python_version:                              3.8.13
build.torch_version:                               1.12.1+cu113
build.env.TORCH_CUDA_ARCH_LIST:                    None
build.env.XFORMERS_BUILD_TYPE:                     None
build.env.XFORMERS_ENABLE_DEBUG_ASSERTIONS:        None
build.env.NVCC_FLAGS:                              None
build.env.XFORMERS_PACKAGE_FROM:                   None
source.privacy:                                    open source
'''

def get_python_version_and_parse():
    # Get the current python version
    get_python_version_args = base_args[:]
    get_python_version_args += [
        'python -V'
    ]
    get_python_version_cmd = (" ").join(get_python_version_args)
    print(get_python_version_cmd)
    version = !$get_python_version_cmd
    version = version[0]
    
    # version = subprocess.run(['python', '-V'], stdout=subprocess.PIPE).stdout.decode().strip()
    version = version.split(' ')[1]
    major, minor, patch = version.split('.')
    # formatted_version = f'cp{major}{minor[0]}{patch[0]}'
    formatted_version = f'cp{major}{minor[0]}'
    return formatted_version


def get_xformers_info():
    output = !python -m xformers.info
    output = '\n'.join(output)
    info = {}
    for line in output.strip().split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            info[key.strip()] = value.strip()
    return info

def check_xformers_info(validate_rules):
    info = get_xformers_info()
    # print(info)
    for key, value in validate_rules.items():
        if info.get(key) != value:
            if isinstance(value, bool):
                if str(value) != info.get(key):
                    return False
            else:
                return False
    return True

def find_xformers_whl_and_install(root):
    py_short_ver = get_python_version_and_parse()
    whl_name = f'xformers*{py_short_ver}*.whl'
    whl_path_list = !find $root -name $whl_name
    whl_path_list = list(filter(lambda x: x.endswith('.whl'), whl_path_list))
    print(whl_path_list)
    if whl_path_list:
        whl_path_list.sort(key=lambda x: x.split("-")[1], reverse=False)  # 按照版本号从高到低排序
        whl_path = whl_path_list[0]  # 取最新版本的 whl 文件
        print(f'【最新版本为】{whl_path}')
        
        install_whl_args = base_args[:]
        install_whl_args += [
            f'pip install {whl_path}'
        ]
        install_whl_cmd = (" ").join(install_whl_args)
        !$install_whl_cmd
        return whl_path
    else:
        return None

validate_rules = {
    'is_triton_available': True,
    'pytorch.cuda': 'available',
}

# 首先判断本机是否已经安装了可以运行的Xformers

if check_xformers_info(validate_rules):
    print('Xformers可以正常运行，无需另外安装或编译')
else:
    # 寻找Xformers预编译wheel，并安装返回列表中的第一个wheel
    whl_path = find_xformers_whl_and_install('/root')

### 从源代码的方式安装Xformers

In [ ]:
# # 安装完再次验证Xformers是否能正常运作：
# if check_xformers_info(validate_rules):
#     print(f'安装后可以正常运行，无需重新编译')
# else:
#     print('需要在本机进行Xformers的编译')
#     # (Optional) Makes the build much faster
#     !pip install ninja
#     # Set TORCH_CUDA_ARCH_LIST if running and building on different GPU types
#     # !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

#     xformers_repo_dir = os.path.join(root_dir,'xformers')

#     if not os.path.exists(xformers_repo_dir):
#         !$proxy &&\
#         cd $root_dir &&\
#         git clone https://github.com/facebookresearch/xformers.git

#     # 记录一次编译所需耗时为 6m 17.51s
#     # !$proxy &&\
#     # cd $root_dir &&\
#     # cd xformers &&\
#     # git show-ref main &&\
#     # git symbolic-ref -q HEAD &&\
#     # git submodule update --init --recursive -q &&\
#     # python setup.py egg_info &&\
#     # cd xformers.egg-info &&\
#     # pip install -r requires.txt &&\
#     # cd .. &&\
#     # git rev-parse HEAD &&\
#     # python setup.py bdist_wheel

#     pip_install_xformers_args = base_args[:]
#     pip_install_xformers_args += [
#         proxy,'&&',
#         f'cd {root_dir}','&&',
#         'cd xformers','&&',
#         'git show-ref main','&&',
#         'git symbolic-ref -q HEAD','&&',
#         'git submodule update --init --recursive -q','&&',
#         'python setup.py egg_info','&&',
#         'cd xformers.egg-info','&&',
#         'pip install -r requires.txt','&&',
#         'cd ..','&&',
#         'git rev-parse HEAD','&&',
#         'python setup.py bdist_wheel'
#     ]

#     pip_install_xformers_cmd = (" ").join(pip_install_xformers_args)
#     print(pip_install_xformers_cmd)
#     !$pip_install_xformers_cmd
    
#     # 安装完成后，找到对应的whl，使用pip进行安装
#     find_xformers_whl_and_install('/root')

### 通过PIP安装triton的最新nightly release

In [ ]:
# pip_install_triton_args = base_args[:]
# pip_install_triton_args += [
#     'pip install -U --pre triton'
# ]

# pip_install_triton_cmd = (" ").join(pip_install_triton_args)
# print(pip_install_triton_cmd)
# !$pip_install_triton_cmd

## <i class="fa fa-download"></i> 安装或更新SD-WebUI本体

In [ ]:
from IPython.display import display,clear_output
clear_output()
import ipywidgets as widgets
from func.env import setProxy,getWebUIDir

debugging=False

alertBtn = widgets.Button(
    value=False,
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='' # (FontAwesome names without the `fa-` prefix)
)


display(alertBtn)
    
if os.path.exists(webUIDir):
    
    alertBtn.description='正在更新...'
    alertBtn.button_style='warning'
    alertBtn.icon='upload'
    
    update_sd_webui_args = base_args[:]
    update_sd_webui_args += [
        proxy,'&&',
        f'cd {webUIDir}','&&',
        'git pull','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        # '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        # 'python -m pip install -r requirements_versions.txt'
    ]
    update_sd_webui_cmd = (" ").join(update_sd_webui_args)
    print(update_sd_webui_cmd)
    !$update_sd_webui_cmd

    if debugging==False:
        # clear_output()
        display(alertBtn)
    
    alertBtn.description='更新完成!'
    alertBtn.button_style='success'
    alertBtn.icon='check'

else:
    alertBtn.description='安装SDWebUI...'
    alertBtn.button_style='info'
    alertBtn.icon='download'
    sd_webui_download_args = base_args[:]
    sd_webui_download_args += [
        proxy,'&&',
        f'cd {root_dir}','&&',
        'git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui',
        '&&',
        'cd stable-diffusion-webui','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        # '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        # 'python -m pip install -r requirements_versions.txt'
    ]
    sd_webui_download_cmd = (" ").join(sd_webui_download_args)
    print(sd_webui_download_cmd)
    !$sd_webui_download_cmd
    
#     r = !source activate &&\
#     conda activate py3.10.6 &&\
#     $proxy &&\
#     cd $webUIDir &&\
#     git pull &&\
#     python -V &&\
#     python -m pip install -r requirements.txt

#     display(r)
    
#     if 'Already up to date.' in r:
#         alertBtn.description='更新完成!'
#         alertBtn.button_style='success'
#         alertBtn.icon='check'
#     else:
#         alertBtn.description='更新失败!'
#         alertBtn.button_style='danger'
#         alertBtn.icon='exclamation'


# clear_output(wait=True)

## <i class="fa fa-language"></i> 安装中文语言包

In [ ]:
from func.extensionsInstaller import install

extURL='https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN'
extFileSize=171055

install(extURL,extFileSize)

## <i class="fa fa-images"></i> 安装图库浏览器

In [ ]:
from func.extensionsInstaller import install

extURL='https://github.com/yfszzx/stable-diffusion-webui-images-browser'
extFileSize=82945

install(extURL,extFileSize)

## 挂载.ckpt或.safetensors到SD WebUI模型文件夹

In [ ]:
import os
from func.checksum import generate_checksum,find_folder

# def find_file(root_dir, target_name):
#     paths = []
#     paths = !find $root_dir -name $target_name
#     print(paths)
    
#     paths.sort(key=len)
#     if paths:
#         return paths
#     else:
#         return None
def create_symlink(source_path, symlink_path, symlink_name):
    # 将软链接路径拼接成完整的文件路径
    symlink_file = f"{symlink_path}/{symlink_name}"

    # 如果软链接文件存在，则先删除它
    if os.path.exists(symlink_file):
        os.remove(symlink_file)

    # 创建软链接
    os.symlink(source_path, symlink_file)

    # 验证软链接是否成功创建
    if os.path.islink(symlink_file):
        print(f"{symlink_file} 是一个符号链接。")
        print(f"\033[1m\033[92m{symlink_file}\033[0m ➜ \033[34m{os.readlink(symlink_file)}\033[0m")
    else:
        print(f"{symlink_file} 不是一个符号链接。")

def find_file(root_dir, target_name):
    paths = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            if filename == target_name:
                path = os.path.join(dirpath, filename)
                paths.append(path)
    paths.sort(key=len)
    if paths:
        return paths
    else:
        return None

def check_model_and_make_symbollink(model_name):
    webUIDir = getWebUIDir()
    sd_models_dir = os.path.realpath(os.path.join(webUIDir,'./models/Stable-diffusion'))
    model_path = os.path.realpath(os.path.join(sd_models_dir,model_name))
    
    for i in [
        '/root/autodl-nas',
        '/root/autodl-fs',
        '/root/autodl-pub',
    ]: # AutoDL中进行nas、fs、pub的搜索
        file_list = find_file(i, model_name)  
        if file_list:
            break
        else:
            continue
            
    if file_list:
        stable_diffusion_v2_dir = file_list[0]

        print(stable_diffusion_v2_dir)

        if stable_diffusion_v2_dir:

            if stable_diffusion_v2_dir==model_path:
                print(f'{model_name}的模型已经在目标位置，无需创建软连接')
            else:
                # !cp $stable_diffusion_v2_dir $sd_models_dir
                # 在Content目录下创建一个指向挂载模型的软连接
                # create_symlink(
                #     source_path = stable_diffusion_v2_dir, # openbayes 特有的目录，是外部数据集挂载到当前镜像的目录，编号从0~4
                #     symlink_path = sd_models_dir, # fast-dreambooth.ipynb的 model download cell一般下载模型的路径
                #     symlink_name = model_name # 模型名称
                # )
                
                !cp $stable_diffusion_v2_dir $models_dir/$model_name
        # else:
        #     print('找不到stable-diffusion-v2的diffuser形态的模型，尝试通过运行convertodiffv2.ipynb来获取模型')
        #     convertodiffv2_path = os.path.join(repo_root_dir,'./tools/model_convert_tools/convertodiffv2.ipynb')
        #     %run $convertodiffv2_path
        #     check_model_and_make_symbollink()
    else:
        print(f'不存在{model_name}')

# check_model_and_make_symbollink('v2-1_768-nonema-pruned.ckpt')
check_model_and_make_symbollink('chilloutmix_NiPrunedFp16.safetensors')

## <i class="fa fa-running"></i> 安装ControlNet插件

In [ ]:
from func.extensionsInstaller import install

extURL='https://github.com/Mikubill/sd-webui-controlnet'
extFileSize=0

install(extURL,extFileSize)

# 在使用segmentation这个preprocessor的时候需要，会遇到报错，通过以下方法进行修复：
# https://github.com/lllyasviel/ControlNet/issues/61#issuecomment-1434005703
# https://github.com/Mikubill/sd-webui-controlnet/issues/70

# 这种方法试过，但不生效

# uninstall_mmcv_args = base_args[:]
# uninstall_mmcv_args += [
#     'pip uninstall mmcv -y'
# ]
# uninstall_mmcv_cmd = (" ").join(uninstall_mmcv_args)
# !$uninstall_mmcv_cmd

#以下这种方法会更改了pythonpath的环境，影响其他库的调用

# import os
# from func.env import getExtDir

# extDir = getExtDir()

# def find_and_set_uniformer_path(dir_name):
#     uniformer_path = None
#     # 遍历指定目录及其子目录下的所有文件和目录
#     for root, dirs, files in os.walk(dir_name):
#         # 如果uniformer是一个目录，检查是否在dirs中
#         if 'uniformer' in dirs:
#             # 获取uniformer所在的完整路径，并打印出来
#             uniformer_path = os.path.join(root, 'uniformer')
#             print(uniformer_path)
#             # 退出循环，不再继续遍历其他目录和文件
#             break

#         # 如果uniformer是一个文件，检查是否在files中
#         if 'uniformer' in files:
#             # 获取uniformer所在的完整路径，并打印出来
#             uniformer_path = os.path.join(root, 'uniformer')
#             print(uniformer_path)
#             # 退出循环，不再继续遍历其他目录和文件
#             break

#     if uniformer_path is not None:
#         # 设置PYTHONPATH环境变量，将uniformer所在的路径添加到当前PYTHONPATH的值中
#         os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + os.pathsep + uniformer_path
#         # 验证环境变量已经设置成功
#         verify_uniformer_path_set(uniformer_path)
#     else:
#         print("在目录{}中未找到uniformer".format(dir_name))

# def verify_uniformer_path_set(uniformer_path):
#     # 获取当前PYTHONPATH环境变量的值，并检查uniformer所在的路径是否在其中
#     python_path = os.environ.get('PYTHONPATH', '')
#     if uniformer_path in python_path.split(os.pathsep):
#         print("PYTHONPATH环境变量设置成功")
#     else:
#         print("PYTHONPATH环境变量设置失败")

# # 调用函数，并传入要搜索的目录名
# find_and_set_uniformer_path(extDir)

## 部署ControlNet模型
解压ControlNet模型并在controlnet拓展的模型目录下创建对应模型的符号链接

In [ ]:
def check_ContorlNet_model_and_make_symbollink(model_name):
    global webUIDir,models_dir
    try:
        webUIDir,models_dir
    except:
        webUIDir = getWebUIDir()
        models_dir = os.path.realpath(os.path.join(webUIDir,'./extensions/sd-webui-controlnet/models'))
        
    model_path = os.path.realpath(os.path.join(models_dir,model_name))
    
    for i in [
        '/root/autodl-nas',
        '/root/autodl-fs',
        '/root/autodl-pub',
    ]: # AutoDL中进行nas、fs、pub的搜索
        file_list = find_file(i, model_name)  
        if file_list:
            break
        else:
            continue

    if file_list:
        stable_diffusion_v2_dir = file_list[0]

        print(stable_diffusion_v2_dir)

        if stable_diffusion_v2_dir:

            if stable_diffusion_v2_dir==model_path:
                print(f'{model_name}的模型已经在目标位置，无需创建软连接')
            else:
                # !cp $stable_diffusion_v2_dir $models_dir
                # 在Content目录下创建一个指向挂载模型的软连接
                
                # create_symlink(
                #     source_path = stable_diffusion_v2_dir, # openbayes 特有的目录，是外部数据集挂载到当前镜像的目录，编号从0~4
                #     symlink_path = models_dir, # fast-dreambooth.ipynb的 model download cell一般下载模型的路径
                #     symlink_name = model_name # 模型名称
                # )
                
                !cp $stable_diffusion_v2_dir $models_dir/$model_name
        # else:
        #     print('找不到stable-diffusion-v2的diffuser形态的模型，尝试通过运行convertodiffv2.ipynb来获取模型')
        #     convertodiffv2_path = os.path.join(repo_root_dir,'./tools/model_convert_tools/convertodiffv2.ipynb')
        #     %run $convertodiffv2_path
        #     check_model_and_make_symbollink()
    else:
        print(f'不存在{model_name}')
        
check_ContorlNet_model_and_make_symbollink('control_canny-fp16.safetensors')
check_ContorlNet_model_and_make_symbollink('control_depth-fp16.safetensors')
check_ContorlNet_model_and_make_symbollink('control_hed-fp16.safetensors')
check_ContorlNet_model_and_make_symbollink('control_mlsd-fp16.safetensors')
check_ContorlNet_model_and_make_symbollink('control_normal-fp16.safetensors')
check_ContorlNet_model_and_make_symbollink('control_openpose-fp16.safetensors')
check_ContorlNet_model_and_make_symbollink('control_scribble-fp16.safetensors')
check_ContorlNet_model_and_make_symbollink('control_seg-fp16.safetensors')

# <i class="fa fa-rocket"></i> 启动Stable-Diffusion-WebUI

In [ ]:
from func.env import setProxy,getWebUIDir,getExtDir,findDir,findFile,getDirSize,styleURL,getArch
import ipywidgets as widgets
from subprocess import getoutput
from func.xformers import checkXformersInstalled
from IPython.display import display,clear_output

extDir=getExtDir()
archCondition = False
canUseXformers = False

cb = getArch()
arch = cb['arch']
gpu = cb['gpu']

whiteList = ['sm80','sm75','sm70','sm86','sm89','sm61']

if arch in whiteList:
    archCondition = True
else:
    archCondition = False
        
if(checkXformersInstalled() and archCondition):
    canUseXformers = True
else:
    canUseXformers = False

canUseXformers = True

webUIDir = getWebUIDir()

args = base_args[:]

args += [
    f'cd {webUIDir}','&&',
    # 'pip install "opencv-python-headless<4.3" ','&&',
    'python launch.py',
    '--share',
    '--opt-split-attention',
    '--port 6006',
    '--deepdanbooru',
    '--administrator',
    '--disable-safe-unpickle',
    # '--no-half'
       ]

try:
    proxy
except NameError:
    print("不存在proxy")
else:
    if region=='未知':
        print("不存在proxy")
    else:
        print(f"已经挂载【{region}】proxy")
        args.insert(0,f"{proxy} && env|grep -i proxy &&")

if(canUseXformers==True):
    print("可以使用Xformers")
    args.append("--xformers")
else:
    print(f"暂不支持【{gpu}】该型号GPU，不使用Xformers")
    
cmd = (" ").join(args)
print(cmd)

!$cmd